In [1]:
import re
import pandas as pd
import numpy as np
import sys
sys.path.append(r"src") #This is the path to my function file
from cleaning import*

In [2]:
#Pandas warned me that sometimes I was overwriting data and this line ignores those warnings :)
pd.options.mode.chained_assignment = None

In [3]:
#Import data
df = pd.read_csv('input/attacks.csv', encoding='cp1252')

In [4]:
#As we explored before, there is a lot of data that is a full row of missing values
df.sample(10)

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
7073,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7335,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23184,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6996,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9719,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24878,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8253,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4276,1955.11.00,Nov-1955,1955.0,Unprovoked,PAPUA NEW GUINEA,"Admiralty Islands, Manus Province","Low Island, Manus",Spearfishing,male,M,...,NaN,"Rapson, p.149",1955.11.00-LowIsland.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,1955.11.00,1955.11.00,2027.0,NaN,NaN
6800,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18392,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#First I drop all the columns that are not relevant to my analysis
df.drop(["Year", "Time", "Species ", 
         "Investigator or Source", "pdf", 
         "href formula", "href", "Case Number.1", 
         "Case Number.2", "original order", "Unnamed: 22", 
         "Unnamed: 23"], axis=1, inplace=True)

In [6]:
#Now I can look at all the columns :)
df.sample(10)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
10203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22388,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4037,1959.12.19.b,19-Dec-1959,Unprovoked,AUSTRALIA,Queensland,"Off Wynnum in Moreton Bay, near Brisbane",Dived from dinghy to retrieve oar in heavy seas,Stanley Arthur Mullen,M,29,FATAL,Y
6551,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25587,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10870,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6899,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22743,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
#I drop all the rows that have all columns filled with trash
df = df[~df.isin([0, np.NAN, '', "NaN", "xx", "0"]).all(1)]

In [8]:
df.sample(10)

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
1191,2008.11.09.b,09-Nov-2008,Boating,AUSTRALIA,South Australia,North Haven,Fishing,NaN,M,NaN,"No injury to occupant, shark bit dinghy & motor",N
5670,1893.04.14,14-Apr-1893,Unprovoked,AUSTRALIA,Queensland,Torres Strait,"Boat capsized, swimming ashore",Kangaroo,M,NaN,Leg bitten,N
252,2016.07.07.a,07-Jul-2016,Boating,USA,California,"Off Palos Verdes peninsula, Los Angeles County",Fishing for sharks,24' boat Shark Tagger Occupant Keith Poe,M,NaN,"No injury. Hull bitten, tooth fragment recovered",N
1433,2006.09.04,04-Sep-2006,Invalid,AUSTRALIA,Queensland,Great Barrier Reef,Diving,Steve Irwin,M,44,"Killed by a stingray, not a shark - a tragedy ...",NaN
6067,1840.09.18.R,Reported 18-Sep-1840,Unprovoked,FIJI,Viti Levu Island,Rewa,NaN,male,M,NaN,Arms & legs severed,N
4906,1935.06.00.b,Jun-1935,Boating,AUSTRALIA,South Australia,Off Port Broughton,NaN,"A dinghy, occupant J.W. Wall",NaN,NaN,Shark bumped dinghy twice,N
1260,2008.05.07.b,07-May-2008,Unprovoked,NEW CALEDONIA,North Province,Hienghène,Fishing,male,M,40,Lower legs bitten,N
502,2014.09.03,03-Sep-2014,Boating,USA,Massachusetts,"Manomet Point, Plymouth, Plymouth County",Kayaking,Ida Parker & Kristen Orr,F,20s,"No injury, shark bit kayak",N
5254,1920.06.27,27-Jun-1920,Provoked,CANADA,Halifax,"Slaunwhite's Ledge, Hubbard Cove",Harpooned shark,occupants: John Chandler & Walter Winters,NaN,NaN,"No injury to occupants, but shark struck boat ...",N
1112,2009.07.11.a,11-Jul-2009,Unprovoked,USA,California,"San Onofre, San Diego County",Paddle-surfing,Brian Hovnanian,M,NaN,"No injury, shark collided with surfer & board",N


In [9]:
#I also drop duplicate rows
df = df.drop_duplicates()
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y


In [10]:
#I drop rows of pseudoduplicates: rows that are almost the same than the next row (9/12 columns or more)
#I do this because I found some pseudosuplicates looking at the data.
df = duplicates(df)

In [11]:
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N)
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N
...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y


In [12]:
#I clean the surf column. If the activity involves surf but it's not written as "Surfing" I just change it to "Surfing"
#This way the data is standarized for later
df["Activity"] = df["Activity"].apply(surf)

In [13]:
#I check the column "Injury" for each entry. If the injury is related to legs, I write "Y" in a new column "Leg Injured"
df["Leg Injured"] = df["Injury"].apply(leg)

In [14]:
df

,Case Number,Date,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Leg Injured
0,2018.06.25,25-Jun-2018,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,N
1,2018.06.18,18-Jun-2018,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,N
2,2018.06.09,09-Jun-2018,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,Y
3,2018.06.08,08-Jun-2018,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,Y
4,2018.06.04,04-Jun-2018,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6297,ND.0005,Before 1903,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,NaN,FATAL,Y,N
6298,ND.0004,Before 1903,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,NaN,FATAL,Y,N
6299,ND.0003,1900-1905,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,NaN,FATAL,Y,N
6300,ND.0002,1883-1889,Unprovoked,PANAMA,NaN,"Panama Bay 8ºN, 79ºW",NaN,Jules Patterson,M,NaN,FATAL,Y,N


In [15]:
#I drop any row that has no information in both injury and activity
df = df[(df['Injury'].notna()) & (df['Activity'].notna())]

In [16]:
#I clean the Age row, but I don't eliminate missing values or values that don't make sense
df["Age"] = df["Age"].apply(age)

In [17]:
df["Age"].unique()

array(['57', '11', '48', 'nan', '18', '52', '15', '12', '32', '10', '21',
       '34', '30', '60', '33', '29', '54', '41', '37', '56', '19', '25',
       '69', '38', '55', '35', '46', '45', '14', '40', '28', '20', '24',
       '26', '49', '22', '7', '31', '17', '13', '42', '3', '8', '50',
       '16', '82', '73', '68', '51', '39', '58', '47', '61', '65', '36',
       '66', '43', '9', '72', '59', '6', '27', '64', '23', '71', '44',
       '62', '63', '70', '53', 'Teen', 'teen', '77', '74', '5', '86',
       'Teens', '84', '\xa0 ', ' ', '6½', '75', '87', '67', 'Elderly',
       'adult', '9 months', '1', '(adult)', 'X', 'MAKE LINE GREEN', '81',
       '"young"', '7 or 8', 'F', 'young', '  ', '78', 'A.M.', '2½'],
      dtype=object)

In [18]:
#I realized that the "Sex" column is spelled "Sex ", so I change it
df = df.rename(columns={'Sex ': 'Sex'})

In [19]:
#I drop Sexes that don't make sense >.<
df.drop(df[(df["Sex"]=="lli") | (df["Sex"]=="N") | (df["Sex"]==".")].index, inplace = True)

In [20]:
#I change "M " to "M"
df["Sex"] = df["Sex"].apply(male)

In [21]:
df["Sex"].unique()

array(['F', 'M', nan], dtype=object)

In [22]:
#Exported data to output for later use
df.to_csv("output/cleaned_data.csv")